In [1]:
import sys
sys.executable

'/usr/bin/python3'

In [2]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket
import glob

import os
import sys
 
import provider
import tf_util
from model import *
print("success")


/home/atas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

success


/home/atas/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/atas/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_

In [4]:
BATCH_SIZE = 12
BATCH_SIZE_EVAL = 12
NUM_POINT = 4096
MAX_EPOCH = 61
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 300000
DECAY_RATE = 0.5

LOG_DIR = 'log'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp model.py %s' % (LOG_DIR)) # bkp of model def
#os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

MAX_NUM_POINT = 4096
NUM_CLASSES = 2

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
#BN_DECAY_DECAY_STEP = float(DECAY_STEP * 2)
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

In [13]:
#Normalize data
xmax = 3.0
xmin = -3.0

DATASET_DIR = 'data/'
f = h5py.File(DATASET_DIR+ 'd0.h5','r')
NUM_FRAMES = f['data'].shape[0]

total_data = np.zeros((6*NUM_FRAMES,4096, 6))
total_label = np.zeros((6*NUM_FRAMES,4096))

for i in range (0,6):
    f = h5py.File(DATASET_DIR+ 'd' +str(i)+'.h5','r')
    data = f['data']
    label = f['label']
    total_data[i*len(data):(i+1)*len(data),:,0:3] = (data[:, :, 0:3] - xmin) / (xmax  - xmin )
    total_data[i*len(data):(i+1)*len(data),:,3:6] = data[:, :, 3:6]
    total_label[i*len(data):(i+1)*len(data),:] = label[:, :]
    
print(total_data.shape)
print(total_label.shape)

(360, 4096, 6)
(360, 4096)


In [14]:
features = ["x","y","z","r","g","b"]
for i in range(6): 
    print(features[i] + "_range :", np.min(total_data[:, :, i]), np.max(total_data[:, :, i]))

x_range : 0.29408538341522217 0.7170508503913879
y_range : 0.317680299282074 0.6883880098660787
z_range : 0.5334503278136253 0.8151378035545349
r_range : 0.0 0.004490316845476627
g_range : 1.5118113878997974e-05 0.004506220109760761
b_range : 0.0 0.004490316845476627


In [15]:
X = total_data
y = total_label

X.shape, y.shape

((360, 4096, 6), (360, 4096))

In [16]:
features = ["x","y","z","r","g","b"]
for i in range(6): 
    print(features[i] + "_range :", np.min(total_data[:, :, i]), np.max(total_data[:, :, i]))

x_range : 0.29408538341522217 0.7170508503913879
y_range : 0.317680299282074 0.6883880098660787
z_range : 0.5334503278136253 0.8151378035545349
r_range : 0.0 0.004490316845476627
g_range : 1.5118113878997974e-05 0.004506220109760761
b_range : 0.0 0.004490316845476627


In [17]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(X, y, test_size=0.1, random_state=42)

print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(324, 4096, 6) (324, 4096)
(36, 4096, 6) (36, 4096)


In [18]:

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def train():
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())
            
            # Note the global_step=batch parameter to minimize. 
            # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
            batch = tf.Variable(0)
            bn_decay = get_bn_decay(batch)
            tf.summary.scalar('bn_decay', bn_decay)

            # Get model and loss 
            pred = get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
            loss = get_loss(pred, labels_pl)
            tf.summary.scalar('loss', loss)

            correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
            accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
            tf.summary.scalar('accuracy', accuracy)

            # Get training operator
            learning_rate = get_learning_rate(batch)
            tf.summary.scalar('learning_rate', learning_rate)
            if OPTIMIZER == 'momentum':
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
            elif OPTIMIZER == 'adam':
                optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.minimize(loss, global_step=batch)
            
            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()
            
        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = True
        sess = tf.Session(config=config)

        # Add summary writers
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

        # Init variables
        init = tf.global_variables_initializer()
        sess.run(init, {is_training_pl:True})

        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'pred': pred,
               'loss': loss,
               'train_op': train_op,
               'merged': merged,
               'step': batch}

        for epoch in range(MAX_EPOCH):
            log_string('**** EPOCH %03d ****' % (epoch))
            sys.stdout.flush()
             
            train_one_epoch(sess, ops, train_writer)
            eval_one_epoch(sess, ops, test_writer)
            
            # Save the variables to disk.
            if epoch % 10 == 0:
                save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
                log_string("Model saved in file: %s" % save_path)



def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    current_data, current_label, _ = provider.shuffle_data(train_data[:,0:NUM_POINT,:], train_label) 
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        if batch_idx % 100 == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred']],
                                         feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
    
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    current_data = test_data[:,0:NUM_POINT,:]
    current_label = np.squeeze(test_label)
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE_EVAL
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE_EVAL
        end_idx = (batch_idx+1) * BATCH_SIZE_EVAL

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}
        summary, step, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['loss'], ops['pred']],
                                      feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE_EVAL*NUM_POINT)
        loss_sum += (loss_val*BATCH_SIZE_EVAL)
        for i in range(start_idx, end_idx):
            for j in range(NUM_POINT):
                l = int(current_label[i, j])
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i-start_idx, j] == l)
            
    log_string('eval mean loss: %f' % (loss_sum / float(total_seen/NUM_POINT)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
         

if __name__ == "__main__":
    train()
    LOG_FOUT.close()



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Tensor("fc2/Relu:0", shape=(12, 128), dtype=float32, device=/device:GPU:0)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.cast` instead.
**** EPOCH 000 ****
----
Current batch/total batch num: 0/27
mean loss: 1.008698
accuracy: 0.798151
----
eval mean loss: 0.241924
eval accuracy: 0.906982
eval avg class acc: 0.756513
Model saved in file: log/model.ckpt
**** EPOCH 001 ****
----
Current batch/total batch num: 0/27
mean loss: 0.196723
accuracy: 0.922791
----
eval mean loss: 0.151227
eval accuracy: 0.932712
eval avg class acc

----
Current batch/total batch num: 0/27
mean loss: 0.045039
accuracy: 0.982717
----
eval mean loss: 0.058332
eval accuracy: 0.975511
eval avg class acc: 0.938893
**** EPOCH 035 ****
----
Current batch/total batch num: 0/27
mean loss: 0.054089
accuracy: 0.977731
----
eval mean loss: 0.063767
eval accuracy: 0.973755
eval avg class acc: 0.945073
**** EPOCH 036 ****
----
Current batch/total batch num: 0/27
mean loss: 0.052845
accuracy: 0.979102
----
eval mean loss: 0.055141
eval accuracy: 0.975525
eval avg class acc: 0.969903
**** EPOCH 037 ****
----
Current batch/total batch num: 0/27
mean loss: 0.044473
accuracy: 0.982811
----
eval mean loss: 0.064377
eval accuracy: 0.971524
eval avg class acc: 0.957181
**** EPOCH 038 ****
----
Current batch/total batch num: 0/27
mean loss: 0.042684
accuracy: 0.984068
----
eval mean loss: 0.103064
eval accuracy: 0.954454
eval avg class acc: 0.866224
**** EPOCH 039 ****
----
Current batch/total batch num: 0/27
mean loss: 0.048192
accuracy: 0.981149
----
